In [2]:
import os
import zipfile
def un_zip(file_name):
    """unzip zip file"""
    zip_file = zipfile.ZipFile(file_name)
    if os.path.isdir(file_name + "_files"):
        pass
    else:
        os.mkdir(file_name + "_files")
    for names in zip_file.namelist():
        zip_file.extract(names,file_name + "_files/")
    zip_file.close()
un_zip('../ensemble-learning/train_modified.zip')

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split

import matplotlib.pylab as plt
%matplotlib inline

In [4]:
train = pd.read_csv('../ensemble-learning/train_modified.zip_files/train_modified.csv')

In [5]:
target='Disbursed' # Disbursed的值就是二元分类的输出
IDcol = 'ID'
train['%s' % target].value_counts()


0    19680
1      320
Name: Disbursed, dtype: int64

In [6]:
x_columns = [x for x in train.columns if x not in [target, IDcol]]
X = train[x_columns]
y = train['%s' % target]

In [13]:
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X,y)
y_pred = gbm0.predict(X)
#按照单行提取(X)[:,1]
y_predprob = gbm0.predict_proba(X)[:,1]
print("Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred))
print("AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob))

IndexError: index 4 is out of bounds for axis 1 with size 2

寻找更优的模型参数

In [24]:
param_test1 = {'n_estimators':range(20,81,10)}
gsearch1 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=300,
                                  min_samples_leaf=20,max_depth=8,max_features='sqrt', subsample=0.8,random_state=10),
                       param_grid = param_test1, scoring='roc_auc',cv=5)
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_
#确定20为最优迭代次数

(<function sklearn.model_selection._search.BaseSearchCV.predict_proba(self, X)>,
 {'n_estimators': 20},
 0.8164836604420731)

In [26]:
#不断修改其他参数以确定最优模型
param_test2 = {'max_depth':range(3,14,2), 'min_samples_split':range(100,801,200)}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, n_estimators=60, min_samples_leaf=20,
      max_features='sqrt', subsample=0.8, random_state=10),
   param_grid = param_test2, scoring='roc_auc', cv=5)
gsearch2.fit(X,y)
gsearch2.best_params_, gsearch2.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'